# Scoring GPAQ

In [3]:
import numpy as np
import pandas as pd
import os


### Import files

In [4]:
path = os.path.join(os.getcwd(),'sample') #insert the path you filled with all .csv files

data_dict = {}

csv_files = [file_name for file_name in os.listdir(path) if file_name.endswith('.csv')]  # list of .csv files
csv_file_count = len(csv_files)

for file_name in os.listdir(path):
    if file_name.endswith('.csv'): #to only import .csv files and avoid any temporal file
        file_path = os.path.join(path, file_name)
        df = pd.read_csv(file_path).T #.T permits a transposition between lignes and columns
        df = df.reset_index(drop = False) #otherwise 'ID' is used as index
            
        #rename columns' names according to first line
        new_cols = df.iloc[0].tolist()  
        df = df.rename(columns=dict(zip(df.columns, new_cols)))  
        df = df.drop(df.index[0]) 
            
        data_dict[file_name] = df

print('Print correct import')
if csv_file_count == len(data_dict):
    print(f"    {csv_file_count} .csv files imported - OK")
else:
    print(f"    Problem: {csv_file_count} .csv files found but only {len(data_dict)} imported")


Print correct import
    2 .csv files imported - OK


### Conditional checks

In [14]:
print("----------")
print('CHECK FOR DUPLICATES')
print('')

names_list = []

for df_name, df in data_dict.items():
    names_list.append(df_name)

if len(names_list) == len(set(names_list)):
    print("No duplicates in dataframe")
else:
    print("Duplicates in dataframe")
    
print("----------")
print('CHECK FOR EVERY YES OR NO ITEMS TO BE FILLED, AND CORRECTLY (1 or 2 value)')
print('')

columns_to_check = ['P1', 'P4', 'P7', 'P10', 'P13'] 

for key, df in data_dict.items():
    mask = (df[columns_to_check] != 1) & (df[columns_to_check] != 2)
    if mask.any().any():
        print(f"Dataframe {key} has invalid values (not 1 or 2) in the following columns:")
        print(df[mask][columns_to_check])
    else:
        pass
    
print("----------")
print('CHECK FOR NO PA BEHAVIOR DESCRIBED IF NO PA MENTIONED BEFORE (NO)')
print('According to ONAPS, questionnaire must be deleated if PA described while NO mentioned')
print('')

for key, df in data_dict.items():
    # check P1
    mask = df['P1'] == 2
    if mask.any():
        sub_df = df.loc[mask, ['P1', 'P2', 'P3a', 'P3b']]
        error_mask = sub_df[['P2', 'P3a', 'P3b']].notnull().any(axis=1)
        if error_mask.any():
            print(f"Error in {key} for columns {sub_df.loc[error_mask].index.tolist()}")
            print(sub_df.loc[error_mask])

    # check P4
    mask = df['P4'] == 2
    if mask.any():
        sub_df = df.loc[mask, ['P4', 'P5', 'P6a', 'P6b']]
        error_mask = sub_df[['P5', 'P6a', 'P6b']].notnull().any(axis=1)
        if error_mask.any():
            print(f"Error in {key} for columns {sub_df.loc[error_mask].index.tolist()}")
            print(sub_df.loc[error_mask])
    
    # check P7
    mask = df['P7'] == 2
    if mask.any():
        sub_df = df.loc[mask, ['P7', 'P8', 'P9a', 'P9b']]
        error_mask = sub_df[['P8', 'P9a', 'P9b']].notnull().any(axis=1)
        if error_mask.any():
            print(f"Error in {key} for columns {sub_df.loc[error_mask].index.tolist()}")
            print(sub_df.loc[error_mask]) 

    # check P10
    mask = df['P10'] == 2
    if mask.any():
        sub_df = df.loc[mask, ['P10', 'P11', 'P12a', 'P12b']]
        error_mask = sub_df[['P11', 'P12a', 'P12b']].notnull().any(axis=1)
        if error_mask.any():
            print(f"Error in {key} for columns {sub_df.loc[error_mask].index.tolist()}")
            print(sub_df.loc[error_mask]) 

    # check P13
    mask = df['P13'] == 2
    if mask.any():
        sub_df = df.loc[mask, ['P13', 'P14', 'P15a', 'P15b']]
        error_mask = sub_df[['P14', 'P15a', 'P15b']].notnull().any(axis=1)
        if error_mask.any():
            print(f"Error in {key} for columns {sub_df.loc[error_mask].index.tolist()}")
            print(sub_df.loc[error_mask])
            
print("----------")
print('CHECK FOR SOME PA BEHAVIOR DESCRIBED IF PA MENTIONED BEFORE (YES)')
print('')

for key, df in data_dict.items():
    cols_to_check = ['P1', 'P4', 'P7', 'P10', 'P13']
    for col in cols_to_check:
        mask = df[col] == 1
        if mask.any():
            col_index = df.columns.get_loc(col)
            next_col = df.columns[col_index+1]
            sub_df = df.loc[mask, [col, next_col]]
            error_mask = (sub_df[col].notnull()) & (sub_df[next_col].isna() | (sub_df[next_col] < 1))
            if error_mask.any():
                print(f"Error in {key} for columns {sub_df.loc[error_mask].index.tolist()}")
                print(sub_df.loc[error_mask])
                
print("----------")
print('CHECK FOR AT LEAST 1 MINUTE OF PA BEHAVIOR DESCRIBED IF PA MENTIONED BEFORE (YES)')
print('According to ONAPS, subdomain must be deleated if no PA described while YES mentioned')
print('')

cols_to_check = ['P1', 'P4', 'P7', 'P10', 'P13']

for key, df in data_dict.items():
    for col in cols_to_check:
        col_index = df.columns.get_loc(col)
        mask = df[col] == 1
        if mask.any():
            sub_df = df.loc[mask, [col, df.columns[col_index+2], df.columns[col_index+3]]]
            error_mask = sub_df[[df.columns[col_index+2], df.columns[col_index+3]]].isna().all(axis=1)
            if error_mask.any():
                print(f"Error in {key} for columns {col} at index {col_index+2} and {col_index+3}")
                print(sub_df.loc[error_mask])
            else:
                sub_df = sub_df.loc[~error_mask]
                error_mask = (sub_df[df.columns[col_index+2]].isna() | sub_df[df.columns[col_index+2]] < 1) & (sub_df[df.columns[col_index+3]].isna() | sub_df[df.columns[col_index+3]] < 1)
                if error_mask.any():
                    print(f"Error in {key} for columns {col} at index {col_index+2} and {col_index+3}")
                    print(sub_df.loc[error_mask])
                    
print("----------")
print('CHECK FOR CORRECT TIME FORMAT: 7 days, 24 hours, 60 minutes')
print('')

columns_to_check = ['P2', 'P5', 'P8', 'P11', 'P14',
                    'P3a', 'P6a', 'P9a', 'P12a', 'P15a', 'P16a',
                    'P3b', 'P6b', 'P9b', 'P12b', 'P15b', 'P16b']

acceptable_ranges = {
    'P2': (0, 7),
    'P5': (0, 7),
    'P8': (0, 7),
    'P11': (0, 7),
    'P14': (0, 7),     
    'P3a': (0, 16), #max authorized according to ONAPS recommandations
    'P6a': (0, 16),
    'P9a': (0, 16),
    'P12a': (0, 16),
    'P15a': (0, 16),
    'P16a': (0, 24), #sedentarity has no maximum (24 hours)
    'P3b': (0, 60),
    'P6b': (0, 60),
    'P9b': (0, 60),
    'P12b': (0, 60),
    'P15b': (0, 60),
    'P16b': (0, 60)
}

aberrant_data = {}

for key, df in data_dict.items():
    for index, row in df.iterrows():
        for col in columns_to_check:
            value = row[col]
            if value < acceptable_ranges[col][0] or value > acceptable_ranges[col][1]:
                if key not in aberrant_data:
                    aberrant_data[key] = []
                aberrant_data[key].append((index, col))
                
for key, values in aberrant_data.items():
    print(f"Dataframe {key}:")
    for index, col in values:
        print(f"Wrong value in {col} at {index}")


----------
CHECK FOR DUPLICATES

No duplicates in dataframe
----------
CHECK FOR EVERY YES OR NO ITEMS TO BE FILLED, AND CORRECTLY (1 or 2 value)

----------
CHECK FOR NO PA BEHAVIOR DESCRIBED IF NO PA MENTIONED BEFORE (NO)
According to ONAPS, questionnaire must be deleated if PA described while NO mentioned

----------
CHECK FOR SOME PA BEHAVIOR DESCRIBED IF PA MENTIONED BEFORE (YES)

----------
CHECK FOR AT LEAST 1 MINUTE OF PA BEHAVIOR DESCRIBED IF PA MENTIONED BEFORE (YES)
According to ONAPS, subdomain must be deleated if no PA described while YES mentioned

----------
CHECK FOR CORRECT TIME FORMAT: 7 days, 24 hours, 60 minutes



#### MET/min/week calculation
We based our calculation on the GPAQ guides and ONAPS recommandations
- VPA_work : vigorous PA realised at work
- MVPA_work : moderate PA realised at work
- travel : PA realised during displacement (considered moderate)
- VPA_hobbies : vigorous PA realised in hobbies
- MVPA_hobbies : moderate PA realised in hobbies
- sed : sedentary time
- work = VPA_work + MVPA_work
- hobbies = VPA_hobbies + MVPA_hobbies
- VPA = VPA_work + VPA_hobbies
- MVPA = MVPA_work + MVPA_hobbies + travel
- PAtot = VPA + MVPA

In [15]:
for key, df in data_dict.items():
    df = df.fillna(0)
    df['VPA_work'] = 8*(df['P2'] * ((df['P3a'] * 60) + df['P3b']))
    df['MVPA_work'] = 4*(df['P5'] * ((df['P6a'] * 60) + df['P6b']))
    df['travel'] = 4*(df['P8'] * ((df['P9a'] * 60) + df['P9b']))
    df['VPA_hobbies'] = 8*(df['P11'] * ((df['P12a'] * 60) + df['P12b']))
    df['MVPA_hobbies'] = 4*(df['P14'] * ((df['P15a'] * 60) + df['P15b']))
    df['sed'] = 7 * ((df['P16a'] * 60) + df['P16b'])
    df['work'] = df['VPA_work'] + df['MVPA_work']
    df['hobbies'] = df['VPA_hobbies']+df['MVPA_hobbies']
    df['VPA'] = df['VPA_work'] + df['VPA_hobbies']
    df['MVPA'] = df['MVPA_work'] + df['MVPA_hobbies'] + df['travel']
    df['PAtot'] = df['VPA'] + df['MVPA']
    data_dict[key] = df


### Saving files

In [16]:
#independant files
saving_path_ind = os.path.join(os.getcwd(),'results') #precise your destination path

for key, value in data_dict.items():
    filename = os.path.join(saving_path_ind, f"{key}")
    value.to_csv(filename)
    
#concatenated files (one unique dataframe)
concatenated_df = pd.concat(data_dict.values(), axis=0)
concatenated_df = concatenated_df.sort_values(by=['ID'])
concatenated_filename = os.path.join(saving_path_ind, "concatenated_data.csv")
concatenated_df.to_csv(concatenated_filename, index=False)
